# Topic Modeling of Amazon Reviews

Import libraries, define functions

In [11]:
import json
import os
import glob
import numpy as np
import random
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

Read data

bash.exe: warning: could not find /tmp, please create!
bash: line 1: ls: command not found


In [19]:
data_dir = 'C:/Users/100048966/Documents/Projects/Topic Modeling/'
# df_reviews = getDF(data_dir+'reviews_Cell_Phones_and_Accessories_5.json.gz')
# df_metadata = getDF(data_dir+'samsung_metadata.json.gz')
df_metadata = getDF(data_dir+'meta_Cell_Phones_and_Accessories.json.gz')

In [3]:
df_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [20]:
df_metadata.head()

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0110400550,"{'also_bought': ['B00C56IXFG', 'B008ZUQWOK', '...",Pink &amp; White 3d Melt Ice-cream Skin Hard C...,3.33,{'Cell Phones & Accessories': 83460},http://ecx.images-amazon.com/images/I/31zn6SOL...,,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Pink & White 3D Melt Ice-Cream Skin Hard Case ...
1,011040047X,"{'buy_after_viewing': ['B008RU7UL2', 'B00698LY...",Purple Hard Case Cover for Iphone 4 4s 4g with...,1.94,{'Cell Phones & Accessories': 495795},http://ecx.images-amazon.com/images/I/41WCZc2d...,NaN,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Purple Hard Case Cover for iPhone 4 4S 4G With...
2,0195866479,"{'buy_after_viewing': ['B00530RXP2', 'B004SH9B...",Hello Kitty Light-weighted Chrome Case Black C...,2.94,{'Cell Phones & Accessories': 371302},http://ecx.images-amazon.com/images/I/41fy1%2B...,NaN,"[[Cell Phones & Accessories, Cases, Basic Cases]]","Thin and light weighted,\nCase's unique design..."
3,0214514706,"{'buy_after_viewing': ['B0042FV2SI', 'B00869D2...",Cool Summer Breeze in the Ocean Beach Collecti...,0.94,{'Cell Phones & Accessories': 778100},http://ecx.images-amazon.com/images/I/415cmp6Q...,NaN,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Product Name: Cool Summer Breeze In The Ocean...
4,0214714705,"{'buy_after_viewing': ['B008EU7HRM', 'B00869D2...",Cool Summer Breeze In The Ocean Beach Collecti...,5.79,{'Cell Phones & Accessories': 654894},http://ecx.images-amazon.com/images/I/41XDwPt2...,NaN,"[[Cell Phones & Accessories, Cases, Basic Cases]]",Product Name: Cool Summer Breeze In The Ocean...


In [32]:
df_metadata[df_metadata['brand'] == 'Samsung']

,asin,related,title,price,salesRank,imUrl,brand,categories,description
103,549980088X,{'buy_after_viewing': ['B001BRHXA6']},Red LED Car/Vehicle Charger for Samsung&copy; ...,3.84,{'Cell Phones & Accessories': 291518},http://ecx.images-amazon.com/images/I/41H0DdRZ...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",Portable lightweight red led car charger offer...
104,5499800650,NaN,Car Charger for Bell Samsung&copy; Instinct Om...,3.88,{'Cell Phones & Accessories': 917469},http://ecx.images-amazon.com/images/I/41RgM6dR...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",Car Charger for Bell Samsung Instinct Omnia i9...
216,8199201460,"{'also_bought': ['B000HD6HRU', 'B000S9DBMW', '...",Samsung Official OEM Car Charger for your SGH-...,3.89,{'Cell Phones & Accessories': 92467},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",NaN
227,8199201940,"{'also_viewed': ['B00CXADUE8', 'B00GB8XIZO', '...",Samsung Official OEM Car Charger for your SPH-...,1.99,{'Cell Phones & Accessories': 611420},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",The Samsung Car Charger gives you the charging...
240,8199201452,"{'also_viewed': ['8199201495'], 'buy_after_vie...",Samsung Official OEM Car Charger for your SGH-...,3.85,{'Cell Phones & Accessories': 593723},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",The Samsung Car Charger gives you the charging...
283,8199201495,"{'also_bought': ['B001ILUWQ2', 'B0031OYCJM', '...",Samsung Official OEM Car Charger for your SGH-...,2.20,{'Cell Phones & Accessories': 211983},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",The Samsung Car Charger gives you the charging...
295,8282002763,"{'also_bought': ['B004YEJBLW', 'B004YTLZ5M', '...",Samsung Official OEM Car Charger for your SGH-...,9.95,{'Cell Phones & Accessories': 35120},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",NaN
296,8282001104,"{'also_bought': ['B00BW0X892', 'B004CP84U2']}",Samsung Official OEM Car Charger for your SCH-...,9.95,{'Cell Phones & Accessories': 106511},http://ecx.images-amazon.com/images/I/31B1s0wX...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",NaN
312,8288852815,NaN,Samsung Official OEM Travel Wall Charger for y...,5.94,{'Cell Phones & Accessories': 341856},http://ecx.images-amazon.com/images/I/31RmPWne...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",Samsung Official OEM Travel Wall Charger for y...
317,8288836925,"{'also_bought': ['B001CZBZOM', 'B008EGWUJQ', '...",Samsung Official OEM Travel Wall Charger for y...,8.78,{'Cell Phones & Accessories': 121782},http://ecx.images-amazon.com/images/I/21ytwsP3...,Samsung,"[[Cell Phones & Accessories, Accessories, Char...",NaN


In [78]:
type(df_metadata['categories'])

pandas.core.series.Series

In [ ]:
for cat in df_metadata['categories']:
    for cat2 in cat:
        print(cat2)

In [167]:
category = [i for cat in df_metadata['categories'] for i in cat ]
print(category[0])
cat_final = [category[i] for i in range(len(category)) if 'Cell Phones' in category[i]]

# for i in range(len(category)):
#     if 'Cell Phones' in category[i]:
#         print(category[i])

cat_final[0]

['Cell Phones & Accessories', 'Cases', 'Basic Cases']


['Cell Phones & Accessories', 'Cell Phones', 'Unlocked Cell Phones']